## Importing the libraries

In [ ]:
import plaidml.keras
plaidml.keras.install_backend()

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab

import statsmodels.api as sm
import statsmodels.tsa.api as smt
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.tsa.stattools as ts
from statsmodels.tsa.stattools import adfuller

from fbprophet import Prophet

import math

# import pyflux as pf

import warnings
warnings.filterwarnings('ignore')

import itertools

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV

from sklearn.model_selection import TimeSeriesSplit

from pandas.plotting import autocorrelation_plot

import re

import sys
import os

from functools import reduce

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM
#from keras.layers import Concatenate
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.utils.data_utils import get_file
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import model_to_dot, plot_model
from keras.datasets import imdb, reuters
from keras.preprocessing import sequence
from keras.optimizers import SGD, RMSprop

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import pickle

from numpy.random import seed

from tensorflow import set_random_seed

In [ ]:
seed(2019)
set_random_seed(2019)

## Helper Functions

In [ ]:
def calc_RMSE(validation_data, prediction_data):
   """
   Calculate RMSE
   """
   a = np.array(validation_data)
   b = np.array(prediction_data)

   return np.sqrt(np.mean((b-a)**2))

In [ ]:
def get_fuller_test(series):
    values = series.values
    result = adfuller(values)
    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))

In [ ]:
def make_plots(data, lags=None):
    '''
    plotting the data with specified number of lags.
    plotting raw data, then ACF and PACF
    '''
    layout = (1, 3)
    raw  = plt.subplot2grid(layout, (0, 0))
    acf  = plt.subplot2grid(layout, (0, 1))
    pacf = plt.subplot2grid(layout, (0, 2))
    
    data.plot(ax = raw, figsize=(12, 6))
    smt.graphics.plot_acf(data, lags = lags, ax = acf)
    smt.graphics.plot_pacf(data, lags = lags, ax = pacf)
    sns.despine()
    plt.tight_layout()

In [ ]:
def make_plots_2(data, lags=None):
    '''
    plotting rolling mean, rolling std and original as per number of lags
    '''
    rolling_mean = data.rolling(window = lags).mean()
    rolling_std = data.rolling(window = lags).std()
    
    original = plt.plot(data, color='black',label = 'Original Timeseries')
    mean = plt.plot(rolling_mean, color='red', label = 'Rolling Mean')
    std = plt.plot(rolling_std, color='orange', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Original, Rolling Mean, Standard Deviation')
    sns.despine()
    plt.show()

In [ ]:
def split_train_test_chronological(df, ratio = 0.9, use_ratio = True, index = 1000):
    
    '''
    Input is a dataframe, and a ratio. Splits dataframe into 2 dataframes chronologically.
    Returns first dataframe up to the index of the length of the input dataframe times the input ratio, 
    and returns second dataframe of remaining elements.
    use_ratio is a flag, wether ratio should be used or indicies instead.
    
    df = input dataframe
    ratio = ratio to be used for splitting
    use_ratio = if True, use ratio, 
    index = index to split input dataframe on
    
    '''
    if use_ratio:
        size = len(df) * ratio
        size_round = round(size)

        df_train = df[0:(size_round)]
        df_test = df[size_round:]
    else:
        df_train = df[0:(index)]
        df_test = df[index:]
    return df_train, df_test

## Importing data

In [ ]:
df_bitcoin_inter = pd.read_pickle('./processed_data/df_bitcoin_inter.pickle')
df_gold_inter = pd.read_pickle('./processed_data/df_gold_inter.pickle')
df_vix_inter = pd.read_pickle('./processed_data/df_vix_inter.pickle')

### Train Test Validation split using helper function

In [ ]:
df_train = df_bitcoin_inter.copy()

In [ ]:
df_train['vix'] = df_vix_inter['y']

In [ ]:
df_train['gold'] = df_gold_inter['y']

In [ ]:
df_train.head(2)

In [ ]:
# df_train_vix, df_test_vix = split_train_test_chronological(df_vix_inter, 0.98)

In [ ]:
# df_train_vix, df_test_vix = split_train_test_chronological(df_vix_inter, 0.98)

In [ ]:
# df_train_vix, df_val_vix = split_train_test_chronological(df_train_vix.reset_index(), 0.98)

In [ ]:
# df_train_vix, df_val_vix = split_train_test_chronological(df_train_vix.reset_index(), 0.98)

## FB Prophet Baseline

In [ ]:
proph = Prophet()

In [ ]:
proph.fit(df_train)

In [ ]:
forecast = proph.predict(df_val)

In [ ]:
proph.plot(forecast)

In [ ]:
proph.plot_components(forecast)

In [ ]:
df_fb = df_train.copy()
df_fb.reset_index(inplace = True)

In [ ]:
df_fb.drop(['index'],axis = 1,inplace = True)

In [ ]:
# df_fb.ds.value_counts()

In [ ]:
df_vix_inter.reset_index(inplace = True)

In [ ]:
df_vix_inter.drop(['index'],axis = 1,inplace = True)

In [ ]:
df_fb['vix'] = df_vix_inter['y']

In [ ]:
proph_with_vix = Prophet()

In [ ]:
proph_with_vix.add_regressor('vix')

In [ ]:
proph_with_vix.fit(df_fb)

In [ ]:
future_with_ex = proph_with_vix.predict(df_val)

## Baseline ARIMA model

In [ ]:
model = ARIMA(df_train.y, order=(1,0,1)).fit()

### Window refitting model

The make_window_refitting_ARIMA_model is taking a really long time to train, as it is retraining for every new observation.

In [ ]:
def make_window_refitting_ARIMA_model(df_t = df_train['y'] , df_v = df_val['y'],p = 1, d = 1, q = 0):
    preds = []
    df_t = list(df_t)

    for i in df_v:
        model = ARIMA(df_t, order=(p,d,q)).fit()
        pred = model.forecast()[0][0]
        preds.append(pred)
        df_t.append(i)

    plt.plot(df_v)
    plt.plot(preds)
    rmse = calc_RMSE(validation,np.array(preds))
    print(rmse)
    
    return rmse,df_v,preds,model

In [ ]:
# res = make_window_refitting_model(df_t = df_train['y'], p = 1, d = 0 , q = 1)

In [ ]:
df_val;

## Baseline LSTM Model

In [ ]:
# date_to_index = pd.Series(index=pd.Index([pd.to_datetime(c) for c in df_train.columns[1:]]),
#                           data=[i for i in range(len(df_train.columns[1:]))])

# series_array = df_train[df_train.columns[1:]].values

In [ ]:
def get_time_block_series(series_array, date_to_index, start_date, end_date):
    
    inds = date_to_index[start_date:end_date]
    return series_array[:,inds]

In [ ]:
def transform_series_encode(series_array):
    
    series_array = np.log1p(np.nan_to_num(series_array)) # filling NaN with 0
    series_mean = series_array.mean(axis=1).reshape(-1,1) 
    series_array = series_array - series_mean
    series_array = series_array.reshape((series_array.shape[0],series_array.shape[1], 1))
    
    return series_array, series_mean

In [ ]:
def transform_series_decode(series_array, encode_series_mean):
    
    series_array = np.log1p(np.nan_to_num(series_array)) # filling NaN with 0
    series_array = series_array - encode_series_mean
    series_array = series_array.reshape((series_array.shape[0],series_array.shape[1], 1))
    
    return series_array

In [ ]:
def split_sequence(sequence, n_steps = 72):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [ ]:
# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

### Standradizing for LSTM

In [ ]:
df_train.head(2)

In [ ]:
df_train.tail(2)

In [ ]:
# load dataset
values = df_train.drop('ds', axis = 1).values
# integer encode direction
# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = StandardScaler()
scaled = scaler.fit_transform(values)
# frame as supervised learning
reframed = series_to_supervised(scaled, 7, 1)


reframed.head()

In [ ]:
# drop columns we don't want to predict
y = reframed.iloc[:,-11].values
X = reframed.drop('var1(t)', axis =1).values
# split into train and test sets

n_train_hours = 12000
n_test_hours = 320
train_X = X[:n_train_hours,:]
train_y = y[:n_train_hours]

val_X= X[n_train_hours:-n_test_hours,]
val_y= y[n_train_hours:-n_test_hours]

test_X = X[-n_test_hours:,:]
test_y = y[-n_test_hours:]



# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
val_X = val_X.reshape((val_X.shape[0], 1, val_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape,val_X.shape, val_y.shape ,test_X.shape, test_y.shape)

In [ ]:
# std = StandardScaler()

In [ ]:
# train = df_train.copy()
# train.head()

In [ ]:
# train.set_index('ds', inplace = True)

In [ ]:
num_steps = 72
# n_features = 3

In [ ]:
#  # train = df_train.copy()

# train = np.array(df_train.y)[:, np.newaxis]
# val = df_val.copy()
# val = np.array(val.y)[:, np.newaxis]
# std.fit(train)

# val = std.transform(val)
# train = std.transform(train)

In [ ]:
# train_vix = np.array(df_train.vix)[:, np.newaxis]

# val_vix = np.array(df_val.vix)[:, np.newaxis]
# std.fit(train_vix)

In [ ]:
# train_gold = np.array(df_train.gold)[:, np.newaxis]

# val_gold = np.array(df_val.gold)[:, np.newaxis]
# std.fit(train_gold)

In [ ]:
# X, y = split_sequence(train, num_steps)

In [ ]:
# X = X.reshape((X.shape[0], X.shape[1], n_features))

In [ ]:
# X.shape

In [ ]:
# X_val, y_val = split_sequence(val, n_steps)

In [ ]:
# X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], n_features))

In [ ]:
LSTM_model_1 = Sequential()
LSTM_model_1.add(LSTM(16, activation='relu', input_shape=(train_X.shape[1], train_X.shape[2])))
# LSTM_model_1.add(Dense(8))
# LSTM_model_1.add(LSTM(16, activation='relu'))
LSTM_model_1.add(Dense(1))
LSTM_model_1.compile(optimizer='adam', loss='mse')

In [ ]:
LSTM_model_1.summary()

In [ ]:
# history = LSTM_model_1.fit(train, train.y, epochs=100, batch_size=128, validation_split=0.25)

In [ ]:
history = LSTM_model_1.fit(train_X, train_y, 
                    epochs= 200, 
                    batch_size = 128, 
                    validation_data=(val_X, val_y),
                    verbose=2,
#                     callbacks=[earlystopper],
                    shuffle=False)

In [ ]:
plt.figure(figsize=(18,8))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

In [ ]:


LSTM_model_2 = Sequential()
LSTM_model_2.add(LSTM(16, activation='relu', input_shape=(train_X.shape[1], train_X.shape[2])))
# LSTM_model_2.add(Dense(8))
# LSTM_model_2.add(LSTM(16, activation='relu'))
LSTM_model_2.add(Dense(1))
LSTM_model_2.compile(optimizer='adam', loss='mse')

In [ ]:
# history_2 = LSTM_model_2.fit(X, y, epochs=100, batch_size=128, validation_data=(X_val, y_val))

In [ ]:
history_2 = LSTM_model_2.fit(train_X, train_y, 
                    epochs= 300, 
                    batch_size = 128, 
                    validation_data=(val_X, val_y),
                    verbose=2,
#                     callbacks=[earlystopper],
                    shuffle=False)

In [ ]:
# LSTM_model_2.save('LSTM_model_2_fit_100epochs.h5')

In [ ]:
keras.models.save_model(
    LSTM_model_2,
    './processed_data/LSTM_model_2_fit_300epochs_2.hdf5',
    overwrite=True,
    include_optimizer=True
)

In [ ]:
LSTM_model_3 = Sequential()
LSTM_model_3.add(LSTM(32, activation='relu', input_shape=(train_X.shape[1], train_X.shape[2]), dropout=0.05,recurrent_dropout=0.05))
LSTM_model_3.add(Dense(32))
# LSTM_model_3.add(LSTM(16, activation='relu'))
LSTM_model_3.add(Dense(1))
LSTM_model_3.compile(optimizer='adam', loss='mse')



In [ ]:
# history_3 = LSTM_model_2.fit(X, y, epochs=300, batch_size=128, validation_data=(X_val, y_val))

In [ ]:
history_3 = LSTM_model_3.fit(train_X, train_y, 
                    epochs= 300, 
                    batch_size = 128, 
                    validation_data=(val_X, val_y),
                    verbose=2,
#                     callbacks=[earlystopper],
                    shuffle=False)

In [ ]:
LSTM_model_3.save('LSTM_model_3_fit_300epochs_with_exogenous.h5')

In [ ]:
keras.models.save_model(
    LSTM_model_3,
    './processed_data/LSTM_model_3_fit_300epochs_with_exogenous.h5',
    overwrite=True,
    include_optimizer=True
)

In [ ]:
train_X.shape[1], train_X.shape[2]